In [11]:
import genbank_tree_funcs

In [12]:
import os
import json
import dendropy
from opentree import OT
from pygbif import occurrences as occ

In [13]:
taxonomy_file = "../../ott3.2/taxonomy.tsv"
fam_synth_node_info_path = 'fam_synth_node_info.json'
#tip_dict_path = 'cluster_tip_info.json'
tip_dict_path = 'tip_dict.json'


assert os.path.exists(taxonomy_file)
assert os.path.exists(fam_synth_node_info_path)
#assert os.path.exists(tip_dict_path)


In [5]:
all_tax_dict = {}
header = ['ottid', 'parent_ottid', 'name','rank','source','uniqname','flags']
for lin in open(taxonomy_file):
        lii=lin.split('\t|\t')
        all_tax_dict[lii[0]]=dict(zip(header, lii))

In [14]:
fams_node_id = json.load(open(fam_synth_node_info_path))

node_ids = {}
for fam in fams_node_id:
    nid = fams_node_id[fam]
    if nid:
        if nid not in node_ids:
            node_ids[nid] = set()
        node_ids[nid].add(fam)

In [15]:
synthtreeid = OT.synth_induced_tree(node_ids = list(node_ids.keys()), label_format="id")
fam_synth_tips = [leaf.taxon.label for leaf in synthtreeid.tree.leaf_node_iter()]

In [16]:
len(fam_synth_tips)

9306

In [13]:
# Download synth tree from https://tree.opentreeoflife.org/about/synthesis-release/v12.3
#tips_over_500 = set()

#fi = '/home/ejmctavish/projects/otapi/opentree12.3_tree/labelled_supertree/labelled_supertree.tre'
#total_synth = dendropy.Tree.get(path=fi, schema="newick")
#for node in total_synth:
#    if len(node.leaf_nodes()) > 500:
##        tips_over_500.add(node.label)
#        synthtreeid_over_500 = OT.synth_induced_tree(node_ids = list(tips_over_500), label_format="id")
#synth_tips_over_500 = [leaf.taxon.label for leaf in synthtreeid_over_500.tree.leaf_node_iter()]
#synth_tips_over_500.sort()
#len(synth_tips_over_500)
#
#with open('synth_tips_over_500.json', 'w') as outfile:
#    json.dump(synth_tips_over_500, outfile)

In [ ]:
synth_tips_over_500 = json.load(open('synth_tips_over_500.json'))

In [ ]:
#node_tips = genbank_tree_funcs.get_tips_for_nodes(synth_tips_over_500)
node_tips = json.load(open('node_tip.json'))

In [ ]:
#with open('node_tip.json', 'w') as outfile:
#    json.dump(node_tips, outfile)

In [ ]:
#tip_dict = json.load(open(tip_dict_path))

In [ ]:
!ls *.json

In [17]:
tip_dict = {}
for i in range(10):
    tmp_dict = json.load(open("tip_dict_{}.json".format(i)))
    tip_dict.update(tmp_dict)
    tmp_dict = json.load(open("cluster_tip_info.json"))
    tip_dict.update(tmp_dict)
    tmp_dict = json.load(open("tip_dict_local.json"))
    tip_dict.update(tmp_dict)

In [22]:
i

9

In [23]:
len(tip_dict)

2391916

In [18]:
node_annotation = genbank_tree_funcs.generate_node_annotations(synth_tips_over_500, node_tips, tip_dict)

NameError: name 'synth_tips_over_500' is not defined

In [ ]:
node_annotation['ott487331']

In [ ]:
genbank_tree_funcs.write_itol_heatmap("genomes_heatmap.txt", "Genome_counts", "Genome_count", node_annotation, 'genomes')
genbank_tree_funcs.write_itol_heatmap("genbank.txt", "genbank_counts", "genbank_count", node_annotation, 'genbank')
genbank_tree_funcs.write_itol_heatmap("GBIF_heatmap.txt", "GBIF_counts", "GBIF_count", node_annotation, 'gbif_occ')
genbank_tree_funcs.write_itol_heatmap("genbank_perc.txt", "genbank_perc", "genbank_perc", node_annotation, 'has_gbif_dat_perc')
genbank_tree_funcs.write_itol_heatmap("GBIF_perc.txt", "GBIF_perc", "GBIF_perc", node_annotation, 'has_nuc_dat_perc')
genbank_tree_funcs.write_itol_heatmap("tips_counted.txt", "Tips_counted", "tips_counted", node_annotation, 'tips_counted')
genbank_tree_funcs.write_itol_heatmap("genome_perc.txt", "genome_perc", "genome_perc", node_annotation, 'has_genome_dat_perc')


In [ ]:
synthtreeid = OT.synth_induced_tree(node_ids = list(synth_tips_over_500), label_format="id")


In [ ]:
synthtreeid.tree.write(path="500_label.tre",schema="newick")

In [ ]:
fi = open('clade_colors.txt','w')
startstr="""TREE_COLORS
#use this template to define branch colors and styles, colored ranges and label colors/font styles/backgrounds
#lines starting with a hash are comments and ignored during parsing

#=================================================================#
#                    MANDATORY SETTINGS                           #
#=================================================================#
#select the separator which is used to delimit the data below (TAB,SPACE or COMMA).This separator must be used throughout this file (except in the SEPARATOR line, which uses space).


SEPARATOR SPACE

DATA
"""
import random
name_labels = ['Bacteria',
               'Archaea',
               'Chloroplastida',
               'Chordata',
               'Lophotrochozoa',
               'Cnidaria',
               'Fungi',
               'SAR',
               'Arthropoda',
               'Echinoidea',
               'Nematoda',
               'Porifera',
               'Excavata']
fi.write(startstr)
for lin in open(taxonomy_file):
        lii=lin.split('\t|\t')
        ott_id = 'ott'+lii[0]
        name = lii[2]
        rank = lii[3]
        if name in name_labels:
            #print(lii)
            #if ott_id in node_annotation:
                r = random.randint(1,255)
                g = random.randint(1,255)
                b = random.randint(1,255)
                color = "rgba({},{},{},0.5)".format(r,g,b)
                fi.write("{} range {} {}\n".format(ott_id, color, name))
        
        

fi.close()

In [ ]:
nodes = list(synth_tips_over_500)
still_missing = []
for nid in nodes:
    for tip in node_tips[nid]:
            if tip not in tip_dict:
                still_missing.append(tip)

In [ ]:
len(still_missing)

In [ ]:
still_missing.sort()

In [ ]:
splits = {}
for i in range(10):
    splits[str(i)] = 0
for item in still_missing:
    splits[item[-1]] += 1

In [ ]:
splits

In [18]:
#fam_node_tips = genbank_tree_funcs.get_tips_for_nodes(fam_synth_tips)
fam_node_tips = json.load(open('fam_node_tip.json'))

In [10]:
nodes = list(fam_node_tips)
still_missing = []
for nid in nodes:
    for tip in fam_node_tips[nid]:
            if tip not in tip_dict:
                still_missing.append(tip)
                
len(still_missing)

0

In [11]:
splits = {}
for i in range(10):
    splits[str(i)] = 0
for item in still_missing:
    splits[item[-1]] += 1

splits

{'0': 0,
 '1': 0,
 '2': 0,
 '3': 0,
 '4': 0,
 '5': 0,
 '6': 0,
 '7': 0,
 '8': 0,
 '9': 0}

In [ ]:
#with open('fam_node_tip.json', 'w') as outfile:
#    json.dump(fam_node_tips, outfile)

In [17]:
fam_node_annotation = genbank_tree_funcs.generate_node_annotations(fam_synth_tips, fam_node_tips, tip_dict)

In [18]:
fam_node_annotation['ott3001061']

{'tips_counted': 34,
 'ncbi_ids': 0,
 'gbif_ids': 34,
 'genomes': 0,
 'genbank': 0,
 'gbif_occ': 34,
 'has_gbif_dat': 17,
 'has_gbif_id': 34,
 'has_nuc_dat': 0,
 'has_genome_dat': 0,
 'has_ncbi_id': 0,
 'total_descendents': 34,
 'has_gbif_dat_perc': 50,
 'has_genome_dat_perc': 0.0,
 'has_nuc_dat_perc': 0,
 'has_gbif_id_perc': 100,
 'has_ncbi_id_perc': 0}

In [14]:
genbank_tree_funcs.write_itol_heatmap("fam_genomes_heatmap.txt", "Genome_counts", "Genome_count", fam_node_annotation, 'genomes')
genbank_tree_funcs.write_itol_heatmap("fam_genbank.txt", "genbank_counts", "genbank_count", fam_node_annotation, 'genbank')
genbank_tree_funcs.write_itol_heatmap("fam_GBIF_heatmap.txt", "GBIF_counts", "GBIF_count", fam_node_annotation, 'gbif_occ')
genbank_tree_funcs.write_itol_heatmap("fam_genbank_perc.txt", "genbank_perc", "genbank_perc", fam_node_annotation, 'has_gbif_dat_perc')
genbank_tree_funcs.write_itol_heatmap("fam_GBIF_perc.txt", "GBIF_perc", "GBIF_perc", fam_node_annotation, 'has_nuc_dat_perc')
genbank_tree_funcs.write_itol_heatmap("fam_tips_counted.txt", "Tips_counted", "tips_counted", fam_node_annotation, 'tips_counted')


In [15]:
genbank_tree_funcs.write_itol_heatmap("fam_genome_perc.txt", "genome_perc", "genome_perc", fam_node_annotation, 'has_genome_dat_perc')

In [31]:
synthtreeid = OT.synth_induced_tree(node_ids = list(fam_synth_tips), label_format="id")


In [19]:
fi = open("families_in_tree.tsv")

fam_nids = set()

for lin in fi:
    lii = lin.strip().split('\t')
    nid = lii[3]
    fam_nids.add(nid)

In [20]:
node

NameError: name 'node' is not defined

In [ ]:
all_fam_node_tips = fam_node_tips

internal_nodes = set()
for node in fam_nids:
    if node not in fam_synth_tips:
        internal_nodes.add(node)
    if node not in all_fam_node_tips:
        synth_sub = OT.synth_subtree(node_id=node, label_format="id")
        if synth_sub.tree:
            sub_tips = [leaf.taxon.label for leaf in synth_sub.tree.leaf_node_iter()]
            all_fam_node_tips[node] = sub_tips
        else:
            pass


In [ ]:
all_fam_node_annotation = genbank_tree_funcs.generate_node_annotations(fam_synth_tips, fam_node_tips, tip_dict)

In [33]:
fi = open("families_in_tree.tsv")
header = fi.readline().split('\t')

new_header = header[:-1]

ofi = open("family_genome_counts.tsv", "w")
new_header.append("Leaf_or_internal")
new_header.append("Count_descendent_tips_opentree")
new_header.append("Count_genomes_in_ncbi")
new_header.append("Percent_tips_w_genomes_in_ncbi")
new_header.append("Notes")

ofi.write("\t".join(new_header))
ofi.write("\n")

for lin in fi:
    lii = lin.strip().split('\t')
    notes = ''
    if len(lii) == 5:
        notes = lii[4]
    lii = lii[:4]
    nid = lii[3]
    if nid in fam_synth_tips:
        lii.append("tip")
    else:
        assert nid in internal_nodes
        lii.append("internal")
    if nid in all_fam_node_annotation:
        lii.append(str(all_fam_node_annotation[nid]['total_descendents']))
        lii.append(str(all_fam_node_annotation[nid].get('genomes', 'NA')))
        lii.append(str(all_fam_node_annotation[nid].get('has_genome_dat_perc','NA')))
    else:
        lii.append("10,000 plus")
        lii.append("not calculated")
        lii.append("not calculated")
    lii.append(notes)
    ofi.write("\t".join(lii))
    ofi.write("\n")

    
    

KeyError: 'mrcaott8837ott9778'

In [20]:
header

['name ', ' ottid ', ' sources ', ' node_id ', ' notes\n']

In [31]:
lii

['Cabiropidae', '3001061', 'worms:248282,gbif:4599,irmng:106236', 'ott3001061']

In [35]:
fam_node_annotation[nid]

KeyError: 'mrcaott8837ott9778'

In [38]:
'mrcaott8837ott9778' in fam_nids

True